In [ ]:
import subprocess

cmd = [
    'python', 'train.py',
    '--img', '1280',
    '--batch', '16',
    '--epochs', '100',
    '--data', 'car.yaml',
    '--cfg', 'models/yolov5n.yaml',
    '--weights', 'yolov5n.pt',
    '--name', 'yolov5n_default'
]

with subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1, universal_newlines=True) as p:
    for line in p.stdout:
        print(line, end='')


In [12]:
import cv2
import torch
import numpy as np

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/yolov5n_default/weights/best.pt')
model.conf = 0.05
model.iou = 0.3

# MOG 초기화
fgbg = cv2.createBackgroundSubtractorMOG2(history=1000, varThreshold=16, detectShadows=False)

# 영상 로드
video_path = './dataset/videos/TalkMedia_talkv_high.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30

roi_y_min = int(height * 0.2)
max_area = width * height * 0.003

# 출력 저장
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_filtered.mp4', fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. YOLO 추론
    results = model(frame)
    dets = results.xyxy[0].cpu().numpy()
    yolo_boxes = []

    for det in dets:
        x1, y1, x2, y2, conf, cls = det
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        w, h = x2 - x1, y2 - y1
        area = w * h
        aspect_ratio = w / h if h != 0 else 0

        if y1 < roi_y_min or area > max_area or aspect_ratio > 5.0:
            continue

        yolo_boxes.append((x1, y1, x2, y2))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 🟩 YOLO 박스

    # 2. MOG 처리
    fgmask = fgbg.apply(frame)
    fgmask = cv2.medianBlur(fgmask, 3)
    fgmask = cv2.dilate(fgmask, None, iterations=1)

    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h if h != 0 else 0
        mx1, my1, mx2, my2 = x, y, x + w, y + h

        if y < roi_y_min or area > max_area or aspect_ratio > 8.0:
            continue

        # 🔵 MOG 박스 표시
        cv2.rectangle(frame, (mx1, my1), (mx2, my2), (255, 0, 0), 2)

        # 🟨 YOLO + MOG 겹침 강조
        for (x1, y1, x2, y2) in yolo_boxes:
            iw = max(0, min(x2, mx2) - max(x1, mx1))
            ih = max(0, min(y2, my2) - max(y1, my1))
            inter = iw * ih
            union = (x2 - x1) * (y2 - y1) + (mx2 - mx1) * (my2 - my1) - inter
            iou = inter / union if union > 0 else 0

            if iou > 0.1:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)  # 🟨 겹치는 YOLO
                break

    # ROI 경계선 시각화 (optional)
    cv2.line(frame, (0, roi_y_min), (width, roi_y_min), (128, 128, 128), 1)

    out.write(frame)
    cv2.imshow("YOLO + MOG Final", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-7-10 Python-3.11.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Use

In [4]:
import cv2
import torch
import numpy as np

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/yolov5n_default/weights/best.pt')
model.conf = 0.05
model.iou = 0.3

# MOG 초기화
fgbg = cv2.createBackgroundSubtractorMOG2(history=1500, varThreshold=12, detectShadows=False)

# 영상 로드
video_path = './dataset/videos/TalkMedia_talkv_high.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 30

# 필터 기준
roi_y_min = int(height * 0.2)
max_area = width * height * 0.003

# 출력 저장
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_mog_strongest.mp4', fourcc, fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. YOLO 추론
    results = model(frame)
    dets = results.xyxy[0].cpu().numpy()
    yolo_boxes = []

    for det in dets:
        x1, y1, x2, y2, conf, cls = det
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        w, h = x2 - x1, y2 - y1
        area = w * h
        aspect_ratio = w / h if h != 0 else 0

        if y1 < roi_y_min or area > max_area or aspect_ratio > 5.0:
            continue

        yolo_boxes.append((x1, y1, x2, y2))
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 🟩 YOLO 박스

    # 2. MOG 후처리 (강화)
    fgmask = fgbg.apply(frame)

    # 전처리: median + 강한 threshold
    fgmask = cv2.medianBlur(fgmask, 5)
    _, fgmask = cv2.threshold(fgmask, 180, 255, cv2.THRESH_BINARY)

    # 후처리: morphology 연산으로 노이즈 제거 & 구멍 메우기
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel_close, iterations=3)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel_open, iterations=2)
    fgmask = cv2.dilate(fgmask, None, iterations=2)

    # 3. MOG 박스 그리기
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mog_boxes = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        aspect_ratio = w / h if h != 0 else 0
        mx1, my1, mx2, my2 = x, y, x + w, y + h

        if y < roi_y_min or area > max_area or aspect_ratio > 8.0:
            continue

        mog_boxes.append((mx1, my1, mx2, my2))
        cv2.rectangle(frame, (mx1, my1), (mx2, my2), (255, 0, 0), 2)  # 🔵 MOG 박스

    # 4. YOLO + MOG 겹침 박스 강조
    for (x1, y1, x2, y2) in yolo_boxes:
        for (mx1, my1, mx2, my2) in mog_boxes:
            iw = max(0, min(x2, mx2) - max(x1, mx1))
            ih = max(0, min(y2, my2) - max(y1, my1))
            inter = iw * ih
            union = (x2 - x1) * (y2 - y1) + (mx2 - mx1) * (my2 - my1) - inter
            iou = inter / union if union > 0 else 0
            if iou > 0.1:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)  # 🟨 겹침 박스
                break

    # 5. 출력
    out.write(frame)
    cv2.imshow("YOLO + Strongest MOG", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-7-10 Python-3.11.13 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
Adding AutoShape... 
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\MOON/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Use